- Building on your previous Python project
- Count the frequency of every word (a word is a sequence of alphanumeric characters, case does NOT matter) in the body of your document
- Write a 64 bit hash function for a word using polynomial rolling hash function
       hash(s) = s[0] + s[1].p  + s[2].

p^2   + ..... + s[n-1].p^(n-1)   mod n       

Here s[i] is the ASCII for letter i in a word, use p = 53 and m = p2^64
Compute Simhash for the document (as shown in slide 57)
Modify your program to take two URLs from the web on the command line, print how many bits are common in their simhashes.



In [1]:

# original = "Gen Gen AI is a predictive language model a translator that sits above existing unstructured data and seeks to generate content that a human would find pleasing. The data sets themselves first need to be rigorously processed and curated Gen, just as data scientists prepare data lakes for advanced analytics and analytical AI"
original = "Tropical fish include fish the the the found in tropical environments around the world, including both freshwater and salt water species"
original = original.lower()

In [2]:
punctuation = ['.', ',', ':', ';',  '/', '\\', '!', '?', '(',')', '[', ']', '{', '}', '|', '&', '^', '%', '<', '>', '~', '#']
def remove_panctuation(original):
    cleaned_data = ""
    for i in original:
        if i not in punctuation:
            cleaned_data +=i
    return cleaned_data
content = remove_panctuation(original)

In [15]:
# def word_frequency(content):
    
#     data = content.split()
#     n = len(data)
#     word_fre_dict = {}
#     for word in range(0,n,2):
#         ngram = " ".join(data[word:word+5]) 
#         if word not in word_fre_dict:
#             word_fre_dict[ngram] = 1
#         else:
#             word_fre_dict[ngram] += 1
#     return word_fre_dict
# word_dict = word_frequency(content)


ngram = 1
n_overlapping = 0
def word_frequency(content):
    data = content.split()
    # print(data)
    n = len(data)
    word_fre_dict = {}
    for word in range(0, n, ngram - n_overlapping):   # steping with ngram - n_overlapping steps for n overlapping
        ngram_data = " ".join(data[word:word+ngram])  # n gram collector
        if ngram_data not in word_fre_dict:
            word_fre_dict[ngram_data] = 1
        else:
            word_fre_dict[ngram_data] = word_fre_dict[ngram_data]+1
            print(word_fre_dict[ngram_data])
    return word_fre_dict

In [16]:
word_frequency(original)

2
2
3
2
4


{'tropical': 2,
 'fish': 2,
 'include': 1,
 'the': 4,
 'found': 1,
 'in': 1,
 'environments': 1,
 'around': 1,
 'world,': 1,
 'including': 1,
 'both': 1,
 'freshwater': 1,
 'and': 1,
 'salt': 1,
 'water': 1,
 'species': 1}

In [5]:
def computeHashValue(word_dict):
    p = 53
    m = 2**8
    words_with_hash = {}
    for word in word_dict:
        hash = 0
        n=len(word)
        for j in range(n):
            ascii_value = ord(word[j])
            hash += ascii_value*(p**j)  # s[n-1].p^(n-1)
        hash = hash % m
        binary_string = bin(hash)[2:]
        padded_binary_string = binary_string.zfill(8)
        words_with_hash[word] = (word_dict[word],padded_binary_string)
    
    return words_with_hash
# computeHashValue(word_dict)

{'tropical fish include fish found': (1, '11111010'),
 'fish include fish found in': (1, '11000111'),
 'include fish found in tropical': (1, '11011111'),
 'fish found in tropical environments': (1, '00010011'),
 'found in tropical environments around': (1, '01111010'),
 'in tropical environments around the': (1, '11111011'),
 'tropical environments around the world': (1, '00000000'),
 'environments around the world including': (1, '10101011'),
 'around the world including both': (1, '01100100'),
 'the world including both freshwater': (1, '10000010'),
 'world including both freshwater and': (1, '10000100'),
 'including both freshwater and salt': (1, '01001000'),
 'both freshwater and salt water': (1, '01110010'),
 'freshwater and salt water species': (1, '10101001'),
 'and salt water species': (1, '00000110'),
 'salt water species': (1, '11101011'),
 'water species': (1, '00010111'),
 'species': (1, '00000100')}

In [58]:
# def compareSimhashes(simhash1,simhash2):
#     commonSimhash = 0
#     unique_count = 0
#     pointer1 = 0
#     pointer2 = 0
#     while pointer1 < len(simhash1) and pointer2 < len(simhash2):
#         if simhash1[pointer1] == simhash2[pointer2]:
#             commonSimhash += 1
#             pointer1 += 1
#             pointer2 += 1
#             unique_count += 1
#         elif simhash1[pointer1] < simhash2[pointer2]:
#             pointer1 += 1
#             unique_count += 1
#         else:
#             pointer2 += 1
#             unique_count += 1
        
#     return f"The Percentage of duplicates is : {(commonSimhash/unique_count)*100} %"

In [62]:
def computeSimHash(original_text):
    content = remove_panctuation(original_text)
    word_dict = word_frequency(content)
    hash64value = computeHashValue(word_dict)
    weight_sum_vector = []
    for i in range(8):
        weight = 0
        for j in hash64value:
            if hash64value[j][1][i] == "1":
                weight += hash64value[j][0]
            else:
                weight -= hash64value[j][0]
        weight_sum_vector.append(weight)
    print(weight_sum_vector)
    fingerprint_vector = [1 if weight > 0 else 0 for weight in weight_sum_vector]
    return fingerprint_vector
computeSimHash(original)

[0, 10, 6, 0, 0, -2, -2, -2]


[0, 1, 1, 0, 0, 0, 0, 0]

In [2]:
# "10101010000000".zfill(45)

'000000000000000000000000000000010101010000000'